In [ ]:
!pip install statsmodels --upgrade
#!pip install greykite

In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import lognorm, norm
from scipy.spatial.distance import pdist, cdist, squareform
from sklearn.gaussian_process import GaussianProcessRegressor as GPR, kernels as gpk
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer, mean_absolute_percentage_error
from statsmodels.tsa.seasonal import seasonal_decompose
from copy import copy, deepcopy
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from copy import copy, deepcopy
import dask
from prophet import Prophet
import zipfile
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
class SpectralMixture(gpk.Kernel):
    def __init__(self, q, w, m, v, d, active_dim=None):
        self.q, self.w, self.m, self.v, self.d = q, w, m, v, d
        self.active_dim = active_dim

    @property
    def anisotropic(self):
        return False

    @property
    def hyperparameter_variance(self):
        return gpk.Hyperparameter("v", "numeric", self.v.ravel(), len(self.v.ravel()))

    @property
    def hyperparameter_mean(self):
        return gpk.Hyperparameter("m", "numeric", self.m.ravel(), len(self.m.ravel()))

    @property
    def hyperparameter_weight(self):
        return gpk.Hyperparameter("w", "numeric", self.w.ravel(), len(self.w.ravel()))

    def __call__(self, X, Y=None, eval_gradient=False):
        w, m, v = self.w[:, np.newaxis], np.reshape(self.m, (self.d, self.q)), np.reshape(self.v, (self.d, self.q))
        assert w.shape == (q, 1), 'Weights must be [q x 1]'
        assert m.shape[1] == q
        assert v.shape[1] == q
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
        tau = X[:, np.newaxis, :] - Y

        # tau(m,n,p) tensordot means(p,q) -> dot_prod(m,n,q)
        # where dot_prod[i,j,k] = tau[i,j]'*means[:,k]
        K = np.cos(2 * np.pi * np.tensordot(tau, m, axes=1)) * \
            np.exp(-2 * np.pi ** 2 * np.tensordot(tau ** 2, v, axes=1))

        # return the weighted sum of the individual
        # Gaussian kernels, dropping the third index
        return np.tensordot(K, w, axes=1).squeeze(axis=(2,))

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return True

    def __repr__(self):
        return "{0}(weight=[{1}], mean=[{2}], variance=[{3}])".format(
            self.__class__.__name__, ", ".join(map("{0:.3g}".format, self.w)),
            ", ".join(map("{0:.3g}".format, self.m)), ", ".join(map("{0:.3g}".format, self.v)))

class PiecewisePolynomial(gpk.Kernel):
    # implemented_q = np.asarray([0,1,2,3])

    def __init__(self, length_scale, q=0, active_dim=None):
        self.q = q
        self.length_scale = length_scale
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'length_scale and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    @property
    def hyperparameter_q(self):
        return gpk.Hyperparameter("q", "numeric", self.q)

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return gpk.Hyperparameter("length_scale", "numeric", self.length_scale, len(self.length_scale))
        return gpk.Hyperparameter("length_scale", "numeric", self.length_scale)

    def fmax(self, r, j, q):
        return np.power(np.maximum(0.0, 1 - r), j + q)

    def get_cov(self, r, j, q):
        if q == 0:
            return 1
        if q == 1:
            return (j + 1) * r + 1
        if q == 2:
            return 1 + (j + 2) * r + ((j ** 2 + 4 * j + 3) / 3.0) * r ** 2
        if q == 3:
            return (
                    1
                    + (j + 3) * r
                    + ((6 * j ** 2 + 36 * j + 45) / 15.0) * r ** 2
                    + ((j ** 3 + 9 * j ** 2 + 23 * j + 15) / 15.0) * r ** 3
            )
        else:
            raise ValueError("Requested kernel q is not implemented.")

    def __call__(self, X, Y=None, eval_gradient=False):
        q = int(np.round(self.q))  # int(self.implemented_q[np.argmin(np.abs(self.implemented_q - q))])
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            r = pdist(X / self.length_scale, metric="cityblock")
            r = squareform(r)
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            r = cdist(X / self.length_scale, Y / self.length_scale, metric="cityblock")
        j = np.floor(X.shape[1] / 2.0) + q + 1
        return self.fmax(r, j, self.q) * self.get_cov(r, j, q)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return True

    def __repr__(self):
        if self.anisotropic:
            return "{0}(q={1:.3g}, length_scale=[{2}])".format(
                self.__class__.__name__, self.q, ", ".join(map("{0:.3g}".format, self.length_scale)))
        else:  # isotropic
            return "{0}(q={1:.3g}, length_scale={2:.3g})".format(
                self.__class__.__name__, self.q, self.length_scale)

class Polynomial(gpk.Kernel):

    def __init__(self, variance=1.0, offset=0.0, degree=1.0, active_dim=None):
        self.degree = degree
        self.variance = variance
        self.offset = offset
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(self.variance), 'variance and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.variance) and len(self.variance) > 1

    @property
    def hyperparameter_periodicity(self):
        return gpk.Hyperparameter("degree", "numeric", self.degree)

    @property
    def hyperparameter_periodicity(self):
        return gpk.Hyperparameter("offset", "numeric", self.offset)

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return gpk.Hyperparameter("variance", "numeric", self.variance, len(self.variance))
        return gpk.Hyperparameter("variance", "numeric", self.variance)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            return (np.matmul(X * self.variance, X.T) + self.offset) ** self.degree
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            return (np.tensordot(X * self.variance, Y, [[-1], [-1]]) + self.offset) ** self.degree

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        if self.anisotropic:
            return "{0}(variance=[{1}], offset={2:.3g}, degree={3:.3g})".format(
                self.__class__.__name__, ", ".join(map("{0:.3g}".format, self.variance)), self.offset, self.degree)
        else:  # isotropic
            return "{0}(variance={1:.3g}, offset={2:.3g}, degree={3:.3g})".format(
                self.__class__.__name__, self.variance, self.offset, self.degree)


class Brownian(gpk.Kernel):

    def __init__(self, variance=1.0, active_dim=None):
        if len(active_dim) != 1:
            raise ValueError("Input dimensional for Brownian kernel must be 1.")
        self.variance = variance
        self.active_dim = active_dim

    @property
    def hyperparameter_variance(self):
        return gpk.Hyperparameter("variance", "numeric", self.variance)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y

        return np.where(np.sign(X) == np.sign(Y.T), self.variance * np.fmin(np.abs(X), np.abs(Y.T)), 0.)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        return "{0}(variance={1:.3g})".format(self.__class__.__name__, self.variance)


class ArcCosine(gpk.Kernel):
    implemented_orders = {0, 1, 2}

    def __init__(self, order=0, variance=1.0, weight_variances=1.0, bias_variance=1.0, active_dim=None):
        if order not in self.implemented_orders:
            raise ValueError("Requested kernel order is not implemented.")
        self.order = order
        self.variance = variance
        self.bias_variance = bias_variance
        self.weight_variances = weight_variances
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.weight_variances), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.weight_variances) and len(self.weight_variances) > 1

    @property
    def hyperparameter_variance(self):
        return gpk.Hyperparameter("variance", "numeric", self.variance)

    @property
    def hyperparameter_weight_variances(self):
        if self.anisotropic:
            return gpk.Hyperparameter("weight_variances", "numeric", self.weight_variances, len(self.weight_variances))
        return gpk.Hyperparameter("weight_variances", "numeric", self.weight_variances)

    @property
    def hyperparameter_bias_variance(self):
        return gpk.Hyperparameter("bias_variance", "numeric", self.bias_variance)

    def _weighted_product(self, X, X2=None):
        if X2 is None:
            return np.sum(self.weight_variances * X ** 2, axis=1) + self.bias_variance
        return np.matmul((self.weight_variances * X), X2.T) + self.bias_variance

    def _J(self, theta):
        """
        Implements the order dependent family of functions defined in equations
        4 to 7 in the reference paper.
        """
        if self.order == 0:
            return np.pi - theta
        elif self.order == 1:
            return np.sin(theta) + (np.pi - theta) * np.cos(theta)
        elif self.order == 2:
            return 3.0 * np.sin(theta) * np.cos(theta) + (np.pi - theta) * (
                    1.0 + 2.0 * np.cos(theta) ** 2)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        X_denominator = np.sqrt(self._weighted_product(X))
        if Y is None:
            Y = X
            Y_denominator = X_denominator
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            Y_denominator = np.sqrt(self._weighted_product(Y))

        numerator = self._weighted_product(X, Y)
        cos_theta = numerator / X_denominator[:, None] / Y_denominator[None, :]
        jitter = 1e-15
        theta = np.arccos(jitter + (1 - 2 * jitter) * cos_theta)

        return self.variance * (1.0 / np.pi) * self._J(theta) * X_denominator[:, None] ** self.order * Y_denominator[
                                                                                                       None,
                                                                                                       :] ** self.order

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        if self.anisotropic:
            return "{0}(variance={1:.3g}, weight_variances=[{2}], bias_variance={3:.3g})".format(
                self.__class__.__name__, self.variance, ", ".join(map("{0:.3g}".format, self.weight_variances)),
                self.bias_variance)
        else:  # isotropic
            return "{0}(variance={1:.3g}, weight_variances={2:.3g}, bias_variance={2:.3g})".format(
                self.__class__.__name__, self.variance, self.weight_variances, self.bias_variance)


class Gibbs(gpk.Kernel):

    def __init__(self, lfunc, args, active_dim=None):
        self.lfunc = lfunc
        self.args = args
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        rx = self.lfunc(X, **self.args)
        if Y is None:
            rz = self.lfunc(X, **self.args)
            dists = squareform(pdist(X, metric='sqeuclidean'))
            np.fill_diagonal(dists, 1)
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            rz = self.lfunc(Y, **self.args)
            dists = cdist(X, Y, metric='sqeuclidean')

        rx2, rz2 = np.reshape(rx ** 2, (-1, 1)), np.reshape(rz ** 2, (1, -1))
        return np.sqrt((2.0 * np.outer(rx, rz)) / (rx2 + rz2)) * np.exp(-1.0 * dists / (rx2 + rz2))

    def diag(self, X):
        return np.alloc(1.0, X.shape[0])

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        if self.anisotropic:
            return "{0}".format(self.__class__.__name__)


class WarpedInput(gpk.Kernel):

    def __init__(self, stationary, func, args, active_dim=None):
        self.stationary = stationary
        self.func = func
        self.args = args
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        X = self.func(X, **self.args)
        if Y is not None:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            Y = self.func(Y, **self.args)

        return self.stationary(X, Y, eval_gradient)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        return ''


class Gabor(gpk.Kernel):

    def __init__(self, stationary, length_scale=1.0, periodicity=1.0, active_dim=None):
        self.stationary = stationary
        self.length_scale = length_scale
        self.periodicity = periodicity
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'length_scale and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    @property
    def hyperparameter_periodicity(self):
        return gpk.Hyperparameter("periodicity", "numeric", self.periodicity)

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return gpk.Hyperparameter("length_scale", "numeric", self.length_scale, len(self.length_scale))
        return gpk.Hyperparameter("length_scale", "numeric", self.length_scale)

    def __call__(self, X, Y=None, eval_gradient=False):
        stationary = self.stationary(length_scale=self.length_scale)
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            dists = squareform(pdist(X / self.length_scale, metric='sqeuclidean'))
            np.fill_diagonal(dists, 1)
            tmp1 = stationary(X, Y, eval_gradient)
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            dists = cdist(X / self.length_scale, Y / self.length_scale, metric='sqeuclidean')
            tmp1 = stationary(X, Y, eval_gradient)

        tmp2 = 2 * np.pi * np.sqrt(dists) * self.length_scale / self.periodicity
        return tmp1 * np.cos(tmp2)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return True

    def __repr__(self):
        if self.anisotropic:
            return "{0}(length_scale=[{1}], periodicity={2:.3g})".format(
                self.__class__.__name__, ", ".join(map("{0:.3g}".format, self.length_scale)), self.periodicity)
        else:  # isotropic
            return "{0}(length_scale={1:.3g}, periodicity={2:.3g})".format(
                self.__class__.__name__, self.length_scale, self.periodicity)


class ConstantKernel(gpk.ConstantKernel):
    def __init__(self, constant_value=1.0, constant_value_bounds=(1e-5, 1e5), active_dim=None):
        super().__init__(constant_value=constant_value, constant_value_bounds=constant_value_bounds)
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class Matern(gpk.Matern):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5), nu=1.5, active_dim=None):
        super().__init__(length_scale=length_scale, length_scale_bounds=length_scale_bounds, nu=nu)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class RationalQuadratic(gpk.RationalQuadratic):
    def __init__(self, length_scale=1.0, alpha=1.0, length_scale_bounds=(1e-05, 100000.0), alpha_bounds=(1e-05, 100000.0),
                 active_dim=None):
        super().__init__(length_scale=length_scale, length_scale_bounds=length_scale_bounds, alpha=alpha, alpha_bounds=alpha_bounds)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class RBF(gpk.RBF):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5), active_dim=None):
        super().__init__(length_scale=length_scale, length_scale_bounds=length_scale_bounds)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class ExpSineSquared(gpk.ExpSineSquared):
    def __init__(self, length_scale=1.0, periodicity=1.0, length_scale_bounds=(1e-5, 1e5),
                 periodicity_bounds=(1e-5, 1e5), active_dim=None):
        super().__init__(length_scale=length_scale, periodicity=periodicity, length_scale_bounds=length_scale_bounds,
                         periodicity_bounds=periodicity_bounds)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class WhiteKernel(gpk.WhiteKernel):
    def __init__(self, noise_level=1.0, noise_level_bounds=(1e-05, 100000.0), active_dim=None):
        super(WhiteKernel, self).__init__(noise_level=noise_level, noise_level_bounds=noise_level_bounds)
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)

In [ ]:
def plot_gp(mu, lb, ub, test_x, test_y, train_x=None, train_y=None, name='', samples={},
            layout='v', xaxis_title='Time', yaxis_title='Sales', fig_size=[1000,500], w=3, f=10):
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Samples"))
    samples = {'sample '+str(i): s for i, s in enumerate(samples)} if not isinstance(samples, dict) else samples
    if train_x is not None:
        fig.add_trace(go.Scatter(x=train_x, y=train_y, mode='lines', name='History', line=dict(width=w), line_color='#1a76ff'))  # plot training data

    fig.add_trace(
        go.Scatter(x=test_x, y=ub, fill=None, mode='lines', line_color='rgba(199, 19, 19, 0.3)',
                   fillcolor='rgba(249, 129, 37, 0.3)', showlegend=True, name='95% uncertainty interval'))
    fig.add_trace(
        go.Scatter(x=test_x, y=lb, fill='tonexty', mode='lines', line_color='rgba(199, 19, 19, 0.3)',
                   fillcolor='rgba(249, 129, 37, 0.3)', showlegend=True, name='95% uncertainty interval'))

    fig.add_trace(go.Scatter(x=test_x, y=mu, line=dict(color='#c71313', width=w), mode='lines', name='Skyolia Forecast'))  # plot the mean
    fig.add_trace(go.Scatter(x=test_x, y=test_y, line=dict(color='#1a76ff', width=w), mode='lines', name='Observed'))
    for k, v in samples.items():
        fig.add_trace(go.Scatter(x=test_x, y=v, name=k, mode='lines',
                                 line=dict(width=w)))  # plot samples
    fig.update_layout(title_text=name, paper_bgcolor='#343434', plot_bgcolor='#343434', xaxis_title=xaxis_title, yaxis_title=yaxis_title,
                          font=dict(family="Montserrat", color="#fff", size=f), title_x=0.5, hovermode="x")
    fig.update_xaxes(showgrid=True, showline=False, gridcolor='#c9c9c9', gridwidth=0.0005)
    fig.update_yaxes(showgrid=True, showline=False, gridcolor='#c9c9c9', gridwidth=0.0005)
    return fig

def date_encoding(df, dt_column, daily=True, weekly=True, yearly=True):
    df[dt_column], periodic_column = df[dt_column].astype('datetime64[ns]'), []
    if daily:
        df['hourofday'] = df[dt_column].dt.hour
        df['sin_hourofday'] = np.sin(2*np.pi*df['hourofday']/np.max(df['hourofday']))
        df['cos_hourofday'] = np.cos(2*np.pi*df['hourofday']/np.max(df['hourofday']))
        df.drop(columns=['hourofday'], inplace=True), periodic_column.extend(['sin_hourofday', 'cos_hourofday'])

    if weekly:
        df['dayofweek'] = df[dt_column].dt.dayofweek
        df['sin_dayofweek'] = np.sin(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
        df['cos_dayofweek'] = np.cos(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
        df.drop(columns=['dayofweek'], inplace=True), periodic_column.extend(['sin_dayofweek', 'cos_dayofweek'])
    if yearly:
        df['dayofyear'] = df[dt_column].dt.dayofyear
        df['sin_dayofyear'] = np.sin(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
        df['cos_dayofyear'] = np.cos(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
        df.drop(columns=['dayofyear'], inplace=True), periodic_column.extend(['sin_dayofyear', 'cos_dayofyear'])
    return df, periodic_column

def timediff_encoding(df, time_col):
    start_date = df[time_col].iloc[0]
    df['delta_t'] = (df[time_col] - start_date) / np.timedelta64(1, 'D')
    df['norm_delta_t'] = df['delta_t']
    return df, ['norm_delta_t'], start_date


def categorical_encoding(train, test, categorical):
    new_cat, OHEncoders = [], {}
    for cat in categorical:
        OE = OneHotEncoder(sparse=False, drop='if_binary')
        train_ohe = OE.fit_transform(train[[cat]])
        test_ohe = OE.transform(test[[cat]])
        for i in range(train_ohe.shape[1]):
            c = OE.categories_[0][i]
            train[cat + '_' + str(c)] = train_ohe[:, i]
            test[cat + '_' + str(c)] = test_ohe[:, i]
            new_cat.append(cat + '_' + str(c))
        OHEncoders[cat] = OE
    return train, test, OHEncoders, new_cat


def numerical_scaling(train, test, numerical):
    MS = MinMaxScaler(feature_range=(0, 1))
    scaled_train = MS.fit_transform(train[numerical])
    scaled_test = MS.transform(test[numerical])
    train[numerical] = scaled_train
    test[numerical] = scaled_test
    return train, test, MS


def output_scaling(train, test, output_col, scaling_type='minmax'):
    YScaler = None
    if scaling_type=='minmax':
        YScaler = MinMaxScaler(feature_range=(0, 1))
        Y_train = YScaler.fit_transform(train[[output_col]]).ravel() + 1e-5
    elif scaling_type=='log':
        Y_train = np.log(train[output_col].values + 1e-5)
    return Y_train, test[output_col].values, YScaler

def gp_ts_pipeline(train, test, categorical, numerical, periodic, scaling_type='minmax'):
    _train, _test = train.copy(), test.copy()
    if len(categorical) > 0:
        _train, _test, OHEncoders, new_cat = categorical_encoding(_train, _test, categorical)  # CATEGORICAL ENCODING
    if len(numerical) > 0:
        _train, _test, MS = numerical_scaling(_train, _test, numerical)  # NUMERCIAL SCALING
    features = new_cat + numerical + periodic
    X_train, T_train = _train[features], _train[time_column]
    X_test, T_test = _test[features], _test[time_column]
    Y_train, Y_test, YScaler = output_scaling(_train, _test, out_column, scaling_type=scaling_type)
    return X_train, T_train, Y_train, X_test, T_test, Y_test, features, OHEncoders, MS, YScaler

def shift_df(df, shift, dropna=True):
    origin = df.copy()
    for i in range(1, shift+1):
        shifted_df = origin.shift(i)
        shifted_df = shifted_df.rename(columns=dict(zip(shifted_df.columns, [str(c)+'_'+str(i) for c in shifted_df.columns])))
        df = pd.concat([shifted_df, df], axis=1)
    return df.dropna() if dropna else df

def plot_cov(covs, cols, subplot_titles, labels=None):
    fig = make_subplots(rows=int(len(covs)/cols) + 1, cols=cols, subplot_titles=subplot_titles)
    height = (1000/cols)*2
    for i, cov in enumerate(covs):
        row, col = int(i / cols)+1, (i%cols)+1
        fig.add_trace(go.Heatmap(z=cov, x=labels, y=labels, colorscale='Greys'), row=row, col=col)
    fig.update_layout(title_text='Cov matrix', height=height)#, yaxis1=dict(domain=[0, 1]), yaxis1=dict(domain=[0, 1])
    return fig

def plot_ts_decomposition(df, index, obs, model="additive", features=False, period=None, samples=None):
    decompose = df[[index, obs]]
    decompose.index = df[index]
    decompose = decompose[[obs]]

    decomposition = seasonal_decompose(decompose, model=model, period=period)
    trend, seasonal, residual = decomposition.trend, decomposition.seasonal, decomposition.resid
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=decompose.index, y=decompose.iloc[:,0], mode='lines', name='observed')) #plot the observed
    fig.add_trace(go.Scatter(x=decompose.index, y=trend.tolist(), mode='lines', name='trend')) #plot the trend
    fig.add_trace(go.Scatter(x=decompose.index, y=seasonal.tolist(), mode='lines', name='seasonal')) #plot the seasonal
    fig.add_trace(go.Scatter(x=decompose.index, y=residual.tolist(), mode='lines', name='residual')) #plot the residual
    if features:
        features = [col for col in list(df.columns) if col not in [index, obs]]
        for col in features:
            fig.add_trace(go.Scatter(x=decompose.index, y=df[col].values, name=col, mode='lines'))
    if samples is not None:
        for i, s in enumerate(samples):
            fig.add_trace(go.Scatter(x=decompose.index, y=s, name='sample '+str(i), mode='lines')) #plot samples
    fig.update_layout(title_text='Decomposition')
    return fig, trend.dropna().values, seasonal.dropna().values, residual.dropna().values

def plot_stl_decomposition(df, index, obs, model="additive", period=None, seasonal=7, samples=None):
    df.index = df[index]
    decompose = df[[index, obs]]
    decompose.index = df[index]
    decompose = decompose[[obs]]

    decomposition = STL(decompose, period=period, seasonal=seasonal).fit()
    trend, seasonal, residual = decomposition.trend, decomposition.seasonal, decomposition.resid
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=decompose.index, y=decompose.iloc[:,0], mode='lines', name='observed')) #plot the observed
    fig.add_trace(go.Scatter(x=decompose.index, y=trend.tolist(), mode='lines', name='trend')) #plot the trend
    fig.add_trace(go.Scatter(x=decompose.index, y=seasonal.tolist(), mode='lines', name='seasonal')) #plot the seasonal
    fig.add_trace(go.Scatter(x=decompose.index, y=residual.tolist(), mode='lines', name='residual')) #plot the residual
    if samples is not None:
        for i, s in enumerate(samples):
            fig.add_trace(go.Scatter(x=decompose.index, y=s, name='sample '+str(i), mode='lines')) #plot samples
    fig.update_layout(title_text='Decomposition')
    return fig, trend.dropna().values, seasonal.dropna().values, residual.dropna().values

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/time series/rossmann sales/sub_rossmann.csv')
df = df.loc[df['Store']==1045]
df

,Store,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Competition
3768,1045,2015-07-31,12216,1385,1,1,0,1,a,c,26990.0,0,1.0
3769,1045,2015-07-30,12042,1329,1,1,0,1,a,c,26990.0,0,1.0
3770,1045,2015-07-29,11185,1307,1,1,0,1,a,c,26990.0,0,1.0
3771,1045,2015-07-28,10901,1314,1,1,0,1,a,c,26990.0,0,1.0
3772,1045,2015-07-27,14919,1629,1,1,0,1,a,c,26990.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4705,1045,2013-01-05,4854,622,1,0,0,0,a,c,26990.0,0,0.0
4706,1045,2013-01-04,6351,769,1,0,0,1,a,c,26990.0,0,0.0
4707,1045,2013-01-03,7582,923,1,0,0,1,a,c,26990.0,0,0.0
4708,1045,2013-01-02,8282,975,1,0,0,1,a,c,26990.0,0,0.0


In [ ]:
pd.DataFrame(data={'Dtypes': df.dtypes, 'Isnull': df.isnull().sum(), 'Nunique': df.nunique()}, index=df.columns)

,Dtypes,Isnull,Nunique
Store,int64,0,1
Date,object,0,942
Sales,int64,0,720
Customers,int64,0,280
Open,int64,0,2
Promo,int64,0,2
StateHoliday,object,0,4
SchoolHoliday,int64,0,2
StoreType,object,0,1
Assortment,object,0,1


In [ ]:
out_column, time_column = 'Sales', 'Date'
df[time_column] = pd.to_datetime(df[time_column])
df.sort_values(by=time_column, inplace = True)

to_remove = ['Customers'] + [c for c in df.columns if df[c].nunique()==1]
features = [c for c in df.columns if (c not in [out_column]) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() >= 2)]
numerical = [c for c in features if (df[c].dtype=='float') or (df[c].dtype=='int') and (c not in categorical)]

fig = make_subplots(rows=1, cols=2, horizontal_spacing = 0.03, subplot_titles=('Label Distribution', "Features Correlation"))
fig.append_trace(go.Histogram(x=df[out_column]), row=1, col=1)
fig.append_trace(go.Heatmap(z=df[numerical+[out_column]].corr(),x=numerical+[out_column],y=numerical+[out_column]), row=1, col=2)
fig.show()

In [ ]:
fig, trend, seasonal, residual = plot_ts_decomposition(df, time_column, out_column, features=True, period=7)
fig.show()
print(np.mean(trend), np.var(trend), np.std(trend))
print(np.mean(seasonal), np.var(seasonal), np.std(seasonal))
print(np.mean(residual), np.var(residual), np.std(residual))

In [ ]:
#df, periodic = date_encoding(df, time_column, daily=False, weekly=True, yearly=True)
df, norm_deltat, start_date = timediff_encoding(df, time_column)

train = df[df[time_column] < '2015-06-14']
test = df[df[time_column]>='2015-06-14']

X_train, T_train, Y_train, X_test, T_test, Y_test, features, OHEncoders, MS, YScaler = gp_ts_pipeline(train, test, categorical, numerical+norm_deltat, ['delta_t'], scaling_type='log')
o_act, t_act, nt_act, f_act = [X_train.columns.get_loc("Open")], [X_train.columns.get_loc("delta_t")], [X_train.columns.get_loc("norm_delta_t")], [X_train.columns.get_loc(c) for c in features if c not in ["norm_delta_t","delta_t","Open"]]
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



((894, 10), (48, 10), (894,), (48,))

In [ ]:
#Store 45

hp = [7.75172145e+02, 1.32441374e-05, 7.00446783e+02, 5.93238373e+02,
       1.41058084e-05, 8.42737251e+02, 1.76468175e+02, 1.42702221e-05,
       1.00000000e-05, 7.92297831e+02, 1.00000000e-05, 1.00000000e-05,
       1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
       1.37187130e+02, 1.82779854e-05, 0.00000000e+00]
def create_kernel(hp):
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    kernel_d = {'Kernel':kernel, 'Poly': poly_comp, 'Week': week_comp, 'Year': year_comp, 'Open':open_comp, 'Trend': trend_comp}
    return kernel, kernel_d

kernel, kernel_d = create_kernel(hp)
cov = kernel(X_train)
mu_test, std = np.full(len(X_train), Y_train[-1]), np.sqrt(np.diag(cov))
samples = np.random.multivariate_normal(mu_test.ravel(), cov, 3)

lb, ub = norm.ppf(0.025, mu_test, std), norm.ppf(0.975, mu_test, std)
fig = plot_gp(mu_test, lb, ub, T_test, Y_test, T_train, Y_train, samples=samples, layout='h')
fig.show()

plot_cov([cov, kernel(X_train, X_test)], 2, '').show()

fig, _, _, _ = plot_ts_decomposition(pd.concat([train, test]), time_column, out_column, samples=samples, period=7)
fig.show()

In [ ]:
#Store 175

hp = [1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 8.98883846e+01,
       9.83115373e-02, 7.45104509e+02, 1.00000000e-05, 1.46407081e+02,
       7.42171297e+02, 1.00000000e-05, 1.23371308e-05, 2.36216866e-05,
       1.23493870e-05, 1.06182264e+02, 2.00577746e-05, 4.11666289e+00,
       2.32734725e-05, 0.00000000e+00]
def create_kernel(hp):
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    kernel_d = {'Kernel':kernel, 'Poly': poly_comp, 'Week': week_comp, 'Year': year_comp, 'Open':open_comp, 'Trend': trend_comp}
    return kernel, kernel_d

kernel, kernel_d = create_kernel(hp)
cov = kernel(X_train)
mu_test, std = np.full(len(X_train), Y_train[-1]), np.sqrt(np.diag(cov))
samples = np.random.multivariate_normal(mu_test.ravel(), cov, 3)

lb, ub = norm.ppf(0.025, mu_test, std), norm.ppf(0.975, mu_test, std)
fig = plot_gp(mu_test, lb, ub, T_test, Y_test, T_train, Y_train, samples=samples, layout='h')
fig.show()

plot_cov([cov, kernel(X_train, X_test)], 2, '').show()

fig, _, _, _ = plot_ts_decomposition(pd.concat([train, test]), time_column, out_column, samples=samples, period=7)
fig.show()

In [ ]:
#Store 224

hp = [1.00000000e+03, 1.00000000e-05, 2.59858454e-05, 4.78313264e+01,
       6.56183373e-05, 1.00000000e-05, 4.49369502e-02, 1.00000000e+03,
       1.38231909e-05, 1.00000000e-05, 5.53066608e-05, 1.00000000e+03,
       7.50198271e-02, 8.03751326e+02, 1.00000000e+03, 1.00000000e+03,
       3.16806238e+01, 1.00000000e+03, 3.00000000e+00]
def create_kernel(hp):
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    kernel_d = {'Kernel':kernel, 'Poly': poly_comp, 'Week': week_comp, 'Year': year_comp, 'Open':open_comp, 'Trend': trend_comp}
    return kernel, kernel_d

kernel, kernel_d = create_kernel(hp)
cov = kernel(X_train)
mu_test, std = np.full(len(X_train), Y_train[-1]), np.sqrt(np.diag(cov))
samples = np.random.multivariate_normal(mu_test.ravel(), cov, 3)

lb, ub = norm.ppf(0.025, mu_test, std), norm.ppf(0.975, mu_test, std)
fig = plot_gp(mu_test, lb, ub, T_test, Y_test, T_train, Y_train, samples=samples, layout='h')
fig.show()

plot_cov([cov, kernel(X_train, X_test)], 2, '').show()

fig, _, _, _ = plot_ts_decomposition(pd.concat([train, test]), time_column, out_column, samples=samples, period=7)
fig.show()

In [ ]:
#Store 337

hp = [3.16401152e-05, 1.00000000e+03, 1.00000000e+03, 1.38408982e+02,
       3.28783741e-05, 3.28621687e-05, 1.00000000e+03, 1.00000000e+03,
       3.31689262e-05, 3.25292224e-05, 3.12872667e-05, 1.00000000e+03,
       1.00000000e+03, 2.93316254e-05, 1.00000000e+03, 3.12332064e-05,
       1.00000000e+03, 1.00000000e+03, 0.00000000e+00]
def create_kernel(hp):
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    kernel_d = {'Kernel':kernel, 'Poly': poly_comp, 'Week': week_comp, 'Year': year_comp, 'Open':open_comp, 'Trend': trend_comp}
    return kernel, kernel_d

kernel, kernel_d = create_kernel(hp)
cov = kernel(X_train)
mu_test, std = np.full(len(X_train), Y_train[-1]), np.sqrt(np.diag(cov))
samples = np.random.multivariate_normal(mu_test.ravel(), cov, 3)

lb, ub = norm.ppf(0.025, mu_test, std), norm.ppf(0.975, mu_test, std)
fig = plot_gp(mu_test, lb, ub, T_test, Y_test, T_train, Y_train, samples=samples, layout='h')
fig.show()

plot_cov([cov, kernel(X_train, X_test)], 2, '').show()

fig, _, _, _ = plot_ts_decomposition(pd.concat([train, test]), time_column, out_column, samples=samples, period=7)
fig.show()

In [ ]:
#Store 1045

hp = [1.00000000e-05, 8.85798772e+02, 5.11183885e+02, 2.87577532e+02,
       1.00000000e-05, 7.16036168e+01, 6.85448127e+02, 1.00000000e+03,
       1.00000000e-05, 2.72945039e-05, 2.02158026e-05, 9.88134463e+02,
       8.95387306e-02, 5.88689473e+02, 6.68679712e+02, 1.00000000e-05,
       6.64019952e+02, 8.84207889e+02, 0.00000000e+00]
def create_kernel(hp):
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    kernel_d = {'Kernel':kernel, 'Poly': poly_comp, 'Week': week_comp, 'Year': year_comp, 'Open':open_comp, 'Trend': trend_comp}
    return kernel, kernel_d

kernel, kernel_d = create_kernel(hp)
cov = kernel(X_train)
mu_test, std = np.full(len(X_train), Y_train[-1]), np.sqrt(np.diag(cov))
samples = np.random.multivariate_normal(mu_test.ravel(), cov, 3)

lb, ub = norm.ppf(0.025, mu_test, std), norm.ppf(0.975, mu_test, std)
fig = plot_gp(mu_test, lb, ub, T_test, Y_test, T_train, Y_train, samples=samples, layout='h')
fig.show()

plot_cov([cov, kernel(X_train, X_test)], 2, '').show()

fig, _, _, _ = plot_ts_decomposition(pd.concat([train, test]), time_column, out_column, samples=samples, period=7)
fig.show()

In [ ]:
def pred_decomposition(kernel_d, scaling_type):
    mus = []
    for i in kernel_d:
        k = kernel_d[i]
        gpr = GPR(kernel=k, optimizer=None, alpha=1e-3).fit(X_train, Y_train)
        mu, std = gpr.predict(X_test, return_std=True)
        if scaling_type=='log':
            mu = np.exp(mu + np.power(std, 2) / 2)
        elif scaling_type=='minmax':
            mu = YScaler.inverse_transform(mu[:,None]).ravel()
        mus.append(mu)
    mus = np.stack(mus)
    mus = dict(zip(kernel_d.keys(), mus))
    return mus

decompose = pred_decomposition(kernel_d, scaling_type='log')
decompose

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:285: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:285: RuntimeWarning:

divide by zero encountered in log



{'Kernel': array([ 3372.89139284,  9924.56478704,  8752.02339409,  8137.39151545,
         8874.43343247,  8460.46037466,  5877.63626681,  3179.28453369,
         8187.95010883,  7443.29099129,  7182.11723591,  8144.56886333,
         8040.70344525,  6679.43814291,  3647.49915492, 10873.88535208,
         9651.00701268,  8988.99338669,  9800.12697797,  9343.53115769,
         6503.97924835,  3533.28097297,  9153.42564219,  8371.93187695,
         8118.31405604,  9234.00129289,  9126.77042885,  7583.1815611 ,
         4143.4125223 , 12379.23681871, 11032.95425559, 10333.80671   ,
        11327.26357823, 10831.43488278,  7526.5658329 ,  4056.93220383,
        11289.3689598 , 10133.24858203,  9625.10327648, 10741.69153058,
        10470.85936242,  7938.1474305 ,  4349.23649491, 13977.43498155,
        12673.28919148, 12114.10630243, 13548.57108935, 13174.77261596]),
 'Poly': array([  610.64759111,  8494.95817184,  8494.95817184,  8494.95817184,
         8494.95817184,  8494.95817184,   61

In [ ]:
#LOG SCALING
gpr = GPR(kernel=kernel, optimizer=None).fit(X_train, Y_train)
ln_mu, ln_std = gpr.predict(X_test, return_std=True)
mu_test = np.exp(ln_mu + np.power(ln_std, 2) / 2)
std_test = np.sqrt((np.exp(np.power(ln_std, 2)) - 1) * np.exp(2 * ln_mu + np.power(ln_std, 2)))
lb, ub = norm.ppf(0.025, mu_test, std_test), norm.ppf(0.975, mu_test, std_test)

pred = np.stack((mu_test, lb, ub)).T
pred = pd.DataFrame(pred, columns=['mu', 'lb', 'ub'])
#samples = np.random.multivariate_normal(mu_test, cov_test, 3)
pred = pd.concat((test.reset_index(drop=True), pred), 1)
fig = plot_gp(pred['mu'].values, pred['lb'].values, pred['ub'].values, T_test, Y_test, T_train, np.exp(Y_train), samples=decompose, layout='h', name='STORE 175')
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:285: RuntimeWarning:

divide by zero encountered in log

<ipython-input-181-b0b5dc848737>:11: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.



In [ ]:
def mase(train_y, test_y, pred):
    n = train_y.shape[0]
    d = np.abs(np.diff(train_y)).sum()/(n-1)
    errors = np.abs(test_y - pred)
    return errors.mean()/d

def mape(test_y, pred):
    return np.round(np.mean(np.abs(100*(test_y-pred)/(test_y + 1e-5))), 0)

def rmspe(test_y, pred):
    return (np.sqrt(np.mean(np.square((test_y - pred) / (test_y + 1e-5))))) * 100

def persistence(train_y, test_y):
    predictions, history = [], list(np.copy(train_y))
    for i in test_y:
        predictions.append(history[-1])
        history.append(i)
    return np.asarray(predictions)

def mda(actual, predicted):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - predicted[:-1])).astype(int))

def wape(true, pred):
    return np.sum(np.abs(true - pred))/np.sum(true)

naive = persistence(Y_train, Y_test)
errors = {'MAE':[mean_absolute_error(Y_test, pred['mu'].values)],
        'RMSE':[mean_squared_error(Y_test, pred['mu'].values)],
        'RMSPE': [rmspe(Y_test, pred['mu'].values)],
        'MAPE':[mean_absolute_percentage_error(Y_test+1e-5, pred['mu'].values)],
        'R2': [r2_score(Y_test, pred['mu'].values)],
        #'MASE':[mase(Y_train, Y_test, pred['mu'].values)],
        'MDA': [mda(Y_test, pred['mu'].values)],
        'WAPE':[wape(Y_test, pred['mu'].values)]}
errors = pd.DataFrame(errors, index =['THIS', 'NAIVE'])
errors

,MAE,RMSE,RMSPE,MAPE,R2,MDA,WAPE
THIS,807.055761,1.107304e+06,12.671889,0.09473,0.845667,0.87234,0.091116
NAIVE,807.055761,1.107304e+06,12.671889,0.09473,0.845667,0.87234,0.091116


In [ ]:
q = np.arange(0.025,1,0.025)
prob_pred = norm.ppf(q[:,None], mu_test, std_test).T
#prob_pred = YScaler.inverse_transform(prob_pred) #minmax scaling
prob_pred = pd.DataFrame(prob_pred, columns=q)

fig = go.Figure()
fig.add_trace(go.Scatter(x=q, y=q, mode='lines', name='IDEAL'))
fig.add_trace(go.Scatter(x=q, y=(prob_pred >= Y_test[:,None]).mean(), mode='lines', name='OBSERVED'))
fig.show()

In [ ]:
-1 * gpr.log_marginal_likelihood_value_

-328.4279707555338

In [ ]:
proph_train = train.rename({time_column:'ds',out_column:'y'},axis='columns')
proph_test = test.rename({time_column:'ds',out_column:'y'},axis='columns')
m = Prophet(interval_width=0.95, yearly_seasonality=True)
for col in [c for c in new_cat+numerical if (c != 'ds') and (c!='y')]:
    m.add_regressor(col)
m.fit(proph_train[['ds']+new_cat+numerical+['y']])

fb_forecast = m.predict(proph_test[['ds']+new_cat+numerical])
fb_forecast

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq27sjm4r/2ca0l47t.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq27sjm4r/wdsjg13c.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34421', 'data', 'file=/tmp/tmpq27sjm4r/2ca0l47t.json', 'init=/tmp/tmpq27sjm4r/wdsjg13c.json', 'output', 'file=/tmp/tmpq27sjm4r/prophet_modeljxjn05tl/prophet_model-20230709141709.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:17:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:17:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Assortment_a,Assortment_a_lower,Assortment_a_upper,Competition,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-06-14,734.779758,1780.026281,6072.231390,734.779758,734.779758,225.485927,225.485927,225.485927,-597.202034,...,-2995.661045,-2995.661045,-2995.661045,224.910962,224.910962,224.910962,0.0,0.0,0.0,3962.526895
1,2015-06-14,734.779758,-1181.400187,3433.551998,734.779758,734.779758,225.485927,225.485927,225.485927,-597.202034,...,-2995.661045,-2995.661045,-2995.661045,224.910962,224.910962,224.910962,0.0,0.0,0.0,1227.930409
2,2015-06-14,734.779758,-1690.676546,2708.349286,734.779758,734.779758,0.000000,0.000000,0.000000,-597.202034,...,-2995.661045,-2995.661045,-2995.661045,224.910962,224.910962,224.910962,0.0,0.0,0.0,569.465288
3,2015-06-14,734.779758,-2199.255434,2218.799742,734.779758,734.779758,0.000000,0.000000,0.000000,-597.202034,...,-2995.661045,-2995.661045,-2995.661045,224.910962,224.910962,224.910962,0.0,0.0,0.0,-35.590395
4,2015-06-14,734.779758,-2438.388706,2187.514011,734.779758,734.779758,225.485927,225.485927,225.485927,-597.202034,...,-2995.661045,-2995.661045,-2995.661045,224.910962,224.910962,224.910962,0.0,0.0,0.0,-52.587152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2015-07-31,775.634984,7366.118157,11857.371672,773.825883,777.453592,225.485927,225.485927,225.485927,-597.202034,...,435.394061,435.394061,435.394061,284.901650,284.901650,284.901650,0.0,0.0,0.0,9499.187336
236,2015-07-31,775.634984,5339.143755,10157.642037,773.818910,777.464329,0.000000,0.000000,0.000000,-597.202034,...,435.394061,435.394061,435.394061,284.901650,284.901650,284.901650,0.0,0.0,0.0,7703.054113
237,2015-07-31,775.634984,4931.240641,9289.001126,773.806168,777.475066,225.485927,225.485927,225.485927,-597.202034,...,435.394061,435.394061,435.394061,284.901650,284.901650,284.901650,0.0,0.0,0.0,7081.001673
238,2015-07-31,775.634984,6071.725821,10803.486509,773.791172,777.485803,225.485927,225.485927,225.485927,-597.202034,...,435.394061,435.394061,435.394061,284.901650,284.901650,284.901650,0.0,0.0,0.0,8393.547801


In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import fbeta_score, make_scorer


result = permutation_importance(gpr, X_test, YScaler.transform(Y_test[:,np.newaxis]).ravel(), n_repeats=10, random_state=42, n_jobs=2, scoring='r2')
fig = go.Figure()
# Use x instead of y argument for horizontal plot
for i in range(X_test.shape[1]):
    fig.add_trace(go.Box(x=result['importances'][i], name=X_test.columns[i]))

fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



In [ ]:
import warnings
import traceback
import dask
import numpy as np
import pandas as pd
import time
from numpy.linalg import LinAlgError
from numpy.linalg import cholesky, det, lstsq, inv
from scipy.spatial.distance import pdist, cdist, squareform
from sklearn.gaussian_process import GaussianProcessRegressor as GPR, kernels as gpk
import category_encoders as ce
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import optimization.numpy_version.single_objective.continuous as co
from dateutil.relativedelta import relativedelta
import datetime
import re

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


class SpectralMixture(gpk.Kernel):
    def __init__(self, q, w, m, v, d, active_dim=None):
        self.q, self.w, self.m, self.v, self.d = q, w, m, v, d
        self.active_dim = active_dim

    @property
    def anisotropic(self):
        return False

    @property
    def hyperparameter_variance(self):
        return gpk.Hyperparameter("v", "numeric", self.v.ravel(), len(self.v.ravel()))

    @property
    def hyperparameter_mean(self):
        return gpk.Hyperparameter("m", "numeric", self.m.ravel(), len(self.m.ravel()))

    @property
    def hyperparameter_weight(self):
        return gpk.Hyperparameter("w", "numeric", self.w.ravel(), len(self.w.ravel()))

    def __call__(self, X, Y=None, eval_gradient=False):
        w, m, v = self.w[:, np.newaxis], np.reshape(self.m, (self.d, self.q)), np.reshape(self.v, (self.d, self.q))
        assert w.shape == (q, 1), 'Weights must be [q x 1]'
        assert m.shape[1] == q
        assert v.shape[1] == q
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
        tau = X[:, np.newaxis, :] - Y

        # tau(m,n,p) tensordot means(p,q) -> dot_prod(m,n,q)
        # where dot_prod[i,j,k] = tau[i,j]'*means[:,k]
        K = np.cos(2 * np.pi * np.tensordot(tau, m, axes=1)) * \
            np.exp(-2 * np.pi ** 2 * np.tensordot(tau ** 2, v, axes=1))

        # return the weighted sum of the individual
        # Gaussian kernels, dropping the third index
        return np.tensordot(K, w, axes=1).squeeze(axis=(2,))

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return True

    def __repr__(self):
        return "{0}(weight=[{1}], mean=[{2}], variance=[{3}])".format(
            self.__class__.__name__, ", ".join(map("{0:.3g}".format, self.w)),
            ", ".join(map("{0:.3g}".format, self.m)), ", ".join(map("{0:.3g}".format, self.v)))


class Polynomial(gpk.Kernel):

    def __init__(self, variance=1.0, offset=0.0, degree=1.0, active_dim=None):
        self.degree = degree
        self.variance = variance
        self.offset = offset
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(self.variance), 'variance and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.variance) and len(self.variance) > 1

    @property
    def hyperparameter_periodicity(self):
        return gpk.Hyperparameter("degree", "numeric", self.degree)

    @property
    def hyperparameter_periodicity(self):
        return gpk.Hyperparameter("offset", "numeric", self.offset)

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return gpk.Hyperparameter("variance", "numeric", self.variance, len(self.variance))
        return gpk.Hyperparameter("variance", "numeric", self.variance)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            return (np.matmul(X * self.variance, X.T) + self.offset) ** self.degree
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            return (np.tensordot(X * self.variance, Y, [[-1], [-1]]) + self.offset) ** self.degree

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        if self.anisotropic:
            return "{0}(variance=[{1}], offset={2:.3g}, degree={3:.3g})".format(
                self.__class__.__name__, ", ".join(map("{0:.3g}".format, self.variance)), self.offset, self.degree)
        else:  # isotropic
            return "{0}(variance={1:.3g}, offset={2:.3g}, degree={3:.3g})".format(
                self.__class__.__name__, self.variance, self.offset, self.degree)


class Brownian(gpk.Kernel):

    def __init__(self, variance=1.0, active_dim=None):
        if len(active_dim) != 1:
            raise ValueError("Input dimensional for Brownian kernel must be 1.")
        self.variance = variance
        self.active_dim = active_dim

    @property
    def hyperparameter_variance(self):
        return gpk.Hyperparameter("variance", "numeric", self.variance)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y

        return np.where(np.sign(X) == np.sign(Y.T), self.variance * np.fmin(np.abs(X), np.abs(Y.T)), 0.)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        return "{0}(variance={1:.3g})".format(self.__class__.__name__, self.variance)


class PiecewisePolynomial(gpk.Kernel):
    # implemented_q = np.asarray([0,1,2,3])

    def __init__(self, length_scale, q=0, active_dim=None):
        self.q = q
        self.length_scale = length_scale
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'length_scale and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    @property
    def hyperparameter_q(self):
        return gpk.Hyperparameter("q", "numeric", self.q)

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return gpk.Hyperparameter("length_scale", "numeric", self.length_scale, len(self.length_scale))
        return gpk.Hyperparameter("length_scale", "numeric", self.length_scale)

    def fmax(self, r, j, q):
        return np.power(np.maximum(0.0, 1 - r), j + q)

    def get_cov(self, r, j, q):
        if q == 0:
            return 1
        if q == 1:
            return (j + 1) * r + 1
        if q == 2:
            return 1 + (j + 2) * r + ((j ** 2 + 4 * j + 3) / 3.0) * r ** 2
        if q == 3:
            return (
                    1
                    + (j + 3) * r
                    + ((6 * j ** 2 + 36 * j + 45) / 15.0) * r ** 2
                    + ((j ** 3 + 9 * j ** 2 + 23 * j + 15) / 15.0) * r ** 3
            )
        else:
            raise ValueError("Requested kernel q is not implemented.")

    def __call__(self, X, Y=None, eval_gradient=False):
        q = int(np.round(self.q))  # int(self.implemented_q[np.argmin(np.abs(self.implemented_q - q))])
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            r = pdist(X / self.length_scale, metric="cityblock")
            r = squareform(r)
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            r = cdist(X / self.length_scale, Y / self.length_scale, metric="cityblock")
        j = np.floor(X.shape[1] / 2.0) + q + 1
        return self.fmax(r, j, self.q) * self.get_cov(r, j, q)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return True

    def __repr__(self):
        if self.anisotropic:
            return "{0}(q={1:.3g}, length_scale=[{2}])".format(
                self.__class__.__name__, self.q, ", ".join(map("{0:.3g}".format, self.length_scale)))
        else:  # isotropic
            return "{0}(q={1:.3g}, length_scale={2:.3g})".format(
                self.__class__.__name__, self.q, self.length_scale)

class ArcCosine(gpk.Kernel):
    implemented_orders = {0, 1, 2}

    def __init__(self, order=0, variance=1.0, weight_variances=1.0, bias_variance=1.0, active_dim=None):
        if order not in self.implemented_orders:
            raise ValueError("Requested kernel order is not implemented.")
        self.order = order
        self.variance = variance
        self.bias_variance = bias_variance
        self.weight_variances = weight_variances
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.weight_variances), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.weight_variances) and len(self.weight_variances) > 1

    @property
    def hyperparameter_variance(self):
        return gpk.Hyperparameter("variance", "numeric", self.variance)

    @property
    def hyperparameter_weight_variances(self):
        if self.anisotropic:
            return gpk.Hyperparameter("weight_variances", "numeric", self.weight_variances, len(self.weight_variances))
        return gpk.Hyperparameter("weight_variances", "numeric", self.weight_variances)

    @property
    def hyperparameter_bias_variance(self):
        return gpk.Hyperparameter("bias_variance", "numeric", self.bias_variance)

    def _weighted_product(self, X, X2=None):
        if X2 is None:
            return np.sum(self.weight_variances * X ** 2, axis=1) + self.bias_variance
        return np.matmul((self.weight_variances * X), X2.T) + self.bias_variance

    def _J(self, theta):
        """
        Implements the order dependent family of functions defined in equations
        4 to 7 in the reference paper.
        """
        if self.order == 0:
            return np.pi - theta
        elif self.order == 1:
            return np.sin(theta) + (np.pi - theta) * np.cos(theta)
        elif self.order == 2:
            return 3.0 * np.sin(theta) * np.cos(theta) + (np.pi - theta) * (
                    1.0 + 2.0 * np.cos(theta) ** 2)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        X_denominator = np.sqrt(self._weighted_product(X))
        if Y is None:
            Y = X
            Y_denominator = X_denominator
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            Y_denominator = np.sqrt(self._weighted_product(Y))

        numerator = self._weighted_product(X, Y)
        cos_theta = numerator / X_denominator[:, None] / Y_denominator[None, :]
        jitter = 1e-15
        theta = np.arccos(jitter + (1 - 2 * jitter) * cos_theta)

        return self.variance * (1.0 / np.pi) * self._J(theta) * X_denominator[:, None] ** self.order * Y_denominator[
                                                                                                       None,
                                                                                                       :] ** self.order

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        if self.anisotropic:
            return "{0}(variance={1:.3g}, weight_variances=[{2}], bias_variance={3:.3g})".format(
                self.__class__.__name__, self.variance, ", ".join(map("{0:.3g}".format, self.weight_variances)),
                self.bias_variance)
        else:  # isotropic
            return "{0}(variance={1:.3g}, weight_variances={2:.3g}, bias_variance={2:.3g})".format(
                self.__class__.__name__, self.variance, self.weight_variances, self.bias_variance)


class Gibbs(gpk.Kernel):

    def __init__(self, lfunc, args, active_dim=None):
        self.lfunc = lfunc
        self.args = args
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        rx = self.lfunc(X, **self.args)
        if Y is None:
            rz = self.lfunc(X, **self.args)
            dists = squareform(pdist(X, metric='sqeuclidean'))
            np.fill_diagonal(dists, 1)
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            rz = self.lfunc(Y, **self.args)
            dists = cdist(X, Y, metric='sqeuclidean')

        rx2, rz2 = np.reshape(rx ** 2, (-1, 1)), np.reshape(rz ** 2, (1, -1))
        return np.sqrt((2.0 * np.outer(rx, rz)) / (rx2 + rz2)) * np.exp(-1.0 * dists / (rx2 + rz2))

    def diag(self, X):
        return np.alloc(1.0, X.shape[0])

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        if self.anisotropic:
            return "{0}".format(self.__class__.__name__)


class WarpedInput(gpk.Kernel):

    def __init__(self, stationary, func, args, active_dim=None):
        self.stationary = stationary
        self.func = func
        self.args = args
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        X = self.func(X, **self.args)
        if Y is not None:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            Y = self.func(Y, **self.args)

        return self.stationary(X, Y, eval_gradient)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return False

    def __repr__(self):
        return ''


class Gabor(gpk.Kernel):

    def __init__(self, stationary, length_scale=1.0, periodicity=1.0, active_dim=None):
        self.stationary = stationary
        self.length_scale = length_scale
        self.periodicity = periodicity
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'length_scale and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    @property
    def hyperparameter_periodicity(self):
        return gpk.Hyperparameter("periodicity", "numeric", self.periodicity)

    @property
    def hyperparameter_length_scale(self):
        if self.anisotropic:
            return gpk.Hyperparameter("length_scale", "numeric", self.length_scale, len(self.length_scale))
        return gpk.Hyperparameter("length_scale", "numeric", self.length_scale)

    def __call__(self, X, Y=None, eval_gradient=False):
        stationary = self.stationary(length_scale=self.length_scale)
        X = np.atleast_2d(X)
        X = X[:, self.active_dim] if self.active_dim is not None else X
        if Y is None:
            dists = squareform(pdist(X / self.length_scale, metric='sqeuclidean'))
            np.fill_diagonal(dists, 1)
            tmp1 = stationary(X, Y, eval_gradient)
        else:
            Y = np.atleast_2d(Y)
            Y = Y[:, self.active_dim] if self.active_dim is not None else Y
            dists = cdist(X / self.length_scale, Y / self.length_scale, metric='sqeuclidean')
            tmp1 = stationary(X, Y, eval_gradient)

        tmp2 = 2 * np.pi * np.sqrt(dists) * self.length_scale / self.periodicity
        return tmp1 * np.cos(tmp2)

    def diag(self, X):
        return np.diag(self(X))

    def is_stationary(self):
        """Returns whether the kernel is stationary. """
        return True

    def __repr__(self):
        if self.anisotropic:
            return "{0}(length_scale=[{1}], periodicity={2:.3g})".format(
                self.__class__.__name__, ", ".join(map("{0:.3g}".format, self.length_scale)), self.periodicity)
        else:  # isotropic
            return "{0}(length_scale={1:.3g}, periodicity={2:.3g})".format(
                self.__class__.__name__, self.length_scale, self.periodicity)


class ConstantKernel(gpk.ConstantKernel):
    def __init__(self, constant_value=1.0, constant_value_bounds=(1e-5, 1e5), active_dim=None):
        super().__init__(constant_value=constant_value, constant_value_bounds=constant_value_bounds)
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class Matern(gpk.Matern):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5), nu=1.5, active_dim=None):
        super().__init__(length_scale=length_scale, length_scale_bounds=length_scale_bounds, nu=nu)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class RationalQuadratic(gpk.RationalQuadratic):
    def __init__(self, length_scale=1.0, alpha=1.0, length_scale_bounds=(1e-05, 100000.0),
                 alpha_bounds=(1e-05, 100000.0),
                 active_dim=None):
        super().__init__(length_scale=length_scale, length_scale_bounds=length_scale_bounds, alpha=alpha,
                         alpha_bounds=alpha_bounds)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class RBF(gpk.RBF):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5), active_dim=None):
        super().__init__(length_scale=length_scale, length_scale_bounds=length_scale_bounds)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class ExpSineSquared(gpk.ExpSineSquared):
    def __init__(self, length_scale=1.0, periodicity=1.0, length_scale_bounds=(1e-5, 1e5),
                 periodicity_bounds=(1e-5, 1e5), active_dim=None):
        super().__init__(length_scale=length_scale, periodicity=periodicity, length_scale_bounds=length_scale_bounds,
                         periodicity_bounds=periodicity_bounds)
        self.active_dim = active_dim
        if active_dim is not None and self.anisotropic:
            assert len(self.active_dim) == len(
                self.length_scale), 'weight_variances and active_dim must have the same length'

    @property
    def anisotropic(self):
        return np.iterable(self.length_scale) and len(self.length_scale) > 1

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)


class WhiteKernel(gpk.WhiteKernel):
    def __init__(self, noise_level=1.0, noise_level_bounds=(1e-05, 100000.0), active_dim=None):
        super(WhiteKernel, self).__init__(noise_level=noise_level, noise_level_bounds=noise_level_bounds)
        self.active_dim = active_dim

    def __call__(self, X, Y=None, eval_gradient=False):
        if self.active_dim == None:
            return super().__call__(X, Y, eval_gradient)
        else:
            X = np.atleast_2d(X)
            X = X[:, self.active_dim]
            if Y is not None:
                Y = np.atleast_2d(Y)
                Y = Y[:, self.active_dim]
            return super().__call__(X, Y, eval_gradient)

def date_encoding(train, test, time_col):
    start_date = train[time_col].iloc[0]
    train['delta_t'] = (train[time_col] - start_date) / np.timedelta64(1, 'D')
    test['delta_t'] = (test[time_col] - start_date) / np.timedelta64(1, 'D')
    train['norm_delta_t'] = train['delta_t']
    test['norm_delta_t'] = test['delta_t']
    return train, test, start_date

def date_encoding(df, dt_column, daily=True, weekly=True, yearly=True):
    df[dt_column], periodic_column = df[dt_column].astype('datetime64[ns]'), []
    if daily:
        df['hourofday'] = df[dt_column].dt.hour
        df['sin_hourofday'] = np.sin(2*np.pi*df['hourofday']/np.max(df['hourofday']))
        df['cos_hourofday'] = np.cos(2*np.pi*df['hourofday']/np.max(df['hourofday']))
        df.drop(columns=['hourofday'], inplace=True), periodic_column.extend(['sin_hourofday', 'cos_hourofday'])

    if weekly:
        df['dayofweek'] = df[dt_column].dt.dayofweek
        df['sin_dayofweek'] = np.sin(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
        df['cos_dayofweek'] = np.cos(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
        df.drop(columns=['dayofweek'], inplace=True), periodic_column.extend(['sin_dayofweek', 'cos_dayofweek'])
    if yearly:
        df['dayofyear'] = df[dt_column].dt.dayofyear
        df['sin_dayofyear'] = np.sin(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
        df['cos_dayofyear'] = np.cos(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
        df.drop(columns=['dayofyear'], inplace=True), periodic_column.extend(['sin_dayofyear', 'cos_dayofyear'])
    return df, periodic_column

def timediff_encoding(df, time_col):
    start_date = df[time_col].iloc[0]
    df['delta_t'] = (df[time_col] - start_date) / np.timedelta64(1, 'D')
    df['norm_delta_t'] = df['delta_t']
    return df, ['norm_delta_t'], start_date


def categorical_encoding(train, test, categorical):
    new_cat, OHEncoders = [], {}
    for cat in categorical:
        OE = OneHotEncoder(sparse=False, drop='if_binary')
        train_ohe = OE.fit_transform(train[[cat]])
        test_ohe = OE.transform(test[[cat]])
        for i in range(train_ohe.shape[1]):
            c = OE.categories_[0][i]
            train[cat + '_' + str(c)] = train_ohe[:, i]
            test[cat + '_' + str(c)] = test_ohe[:, i]
            new_cat.append(cat + '_' + str(c))
        OHEncoders[cat] = OE
    return train, test, OHEncoders, new_cat

def catboost_encoding(train, test, categorical, output_col):
    encoder = ce.CatBoostEncoder(verbose=1, cols=categorical)
    encoder.fit(train[categorical], train[output_col])
    tmp_train, tmp_test = encoder.transform(train[categorical]), encoder.transform(test[categorical])
    new_cat = ['cbe_' + col for col in categorical]
    train[new_cat] = tmp_train
    test[new_cat] = tmp_test
    return train, test, encoder, new_cat

def numerical_scaling(train, test, numerical):
    MS = MinMaxScaler(feature_range=(0, 1))
    scaled_train = MS.fit_transform(train[numerical])
    scaled_test = MS.transform(test[numerical])
    train[numerical] = scaled_train
    test[numerical] = scaled_test
    return train, test, MS

def output_scaling(train, test, output_col, scaling_type='minmax'):
    YScaler = None
    if scaling_type=='minmax':
        YScaler = MinMaxScaler(feature_range=(0, 1))
        Y_train = YScaler.fit_transform(train[[output_col]]).ravel() + 1e-5
    elif scaling_type=='log':
        Y_train = np.log(train[output_col].values + 1e-5)
    return Y_train, test[output_col].values, YScaler

def gp_ts_pipeline(train, test, categorical, numerical, periodic, scaling_type='minmax'):
    _train, _test = train.copy(), test.copy()
    if len(categorical) > 0:
        _train, _test, OHEncoders, new_cat = categorical_encoding(_train, _test, categorical)  # CATEGORICAL ENCODING
    if len(numerical) > 0:
        _train, _test, MS = numerical_scaling(_train, _test, numerical)  # NUMERCIAL SCALING
    features = new_cat + numerical + periodic
    X_train, T_train = _train[features], _train[time_column]
    X_test, T_test = _test[features], _test[time_column]
    Y_train, Y_test, YScaler = output_scaling(_train, _test, out_column, scaling_type=scaling_type)
    return X_train, T_train, Y_train, X_test, T_test, Y_test, features, OHEncoders, MS, YScaler

def shift_df(df, shift, dropna=True, sign=1):
    origin = df.copy()
    pref = 'next_' if sign==-1 else 'prev_'
    for i in range(1, shift+1):
        shifted_df = origin.shift(i*sign)
        shifted_df = shifted_df.rename(columns=dict(zip(shifted_df.columns,
                                                        [pref+str(c)+'_'+str(i) for c in shifted_df.columns])))
        df = pd.concat([df, shifted_df], axis=1)
    return df.dropna() if dropna else df

def create_kernel_1(hp):
    hp[len(f_act) + 7] = deg[np.argmin(np.abs(deg - hp[len(f_act) + 7]))]
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 7], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 3], hp[len(f_act) + 4], hp[len(f_act) + 5], active_dim=nt_act)
    kernel = (poly_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 6])) * open_comp
    return kernel
def create_kernel_2(hp):
    hp[len(f_act) + 11] = deg[np.argmin(np.abs(deg - hp[len(f_act) + 11]))]
    poly_comp = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    return kernel
def create_kernel_3(hp):
    hp[len(f_act) + 11] = nus[np.argmin(np.abs(nus - hp[len(f_act) + 11]))]
    poly_comp = Matern(hp[:len(f_act)], nu=hp[len(f_act) + 11], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)  # nu=hp[len(features)]
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    return kernel
def create_kernel_4(hp):
    poly_comp = RBF(hp[:len(f_act)], active_dim=f_act) * ConstantKernel(hp[len(f_act)], active_dim=f_act)
    open_comp = RBF(hp[len(f_act) + 1], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 2], active_dim=o_act)
    week_comp = ExpSineSquared(hp[len(f_act) + 3], periodicity=7, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 4], active_dim=t_act)
    year_comp = ExpSineSquared(hp[len(f_act) + 5], periodicity=364, active_dim=t_act) * ConstantKernel(hp[len(f_act) + 6], active_dim=t_act)
    trend_comp = ArcCosine(2, hp[len(f_act) + 7], hp[len(f_act) + 8], hp[len(f_act) + 9], active_dim=nt_act)
    kernel = (poly_comp + year_comp + week_comp + trend_comp + gpk.WhiteKernel(hp[len(f_act) + 10])) * open_comp
    return kernel

def create_kernel_5(hp):
    hp[len(features)] = nus[np.argmin(np.abs(nus - hp[len(features)]))]
    poly = Matern(hp[:len(features)], nu=hp[len(features)]) * ConstantKernel(hp[len(features)+1])#nu=hp[len(features)]
    #poly = ArcCosine(2, hp[len(features)], hp[:len(features)], hp[len(features) + 1])
    '''hp[len(features)] = deg[np.argmin(np.abs(deg - hp[len(features)]))]
    poly = PiecewisePolynomial(hp[:len(features)], q=hp[len(features)]) * ConstantKernel(hp[len(features) + 1])'''
    kernel = poly + gpk.WhiteKernel(hp[len(features) + 2])
    return kernel

def create_kernel_6(hp):
    hp[len(f_act)] = deg[np.argmin(np.abs(deg - hp[len(f_act)]))]
    poly = PiecewisePolynomial(hp[:len(f_act)], q=hp[len(f_act)], active_dim=f_act) * ConstantKernel(hp[len(f_act) + 1], active_dim=f_act)
    open = RBF(hp[len(f_act) + 2], active_dim=o_act) * ConstantKernel(hp[len(f_act) + 3], active_dim=o_act)
    kernel = (poly + gpk.WhiteKernel(hp[len(f_act) + 4]))*open
    return kernel

df = pd.read_csv('/home/youcef/Downloads/sub_rossmann.csv')
df = df.loc[df['Store']==1045]
########################################################################################################################
out_column, time_column = 'Sales', 'Date'
df[time_column] = pd.to_datetime(df[time_column])
df.sort_values(by=time_column, inplace = True)
to_remove = ['Customers'] + [c for c in df.columns if df[c].nunique()==1]
features = [c for c in df.columns if (c not in [out_column]) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() >= 2)]
numerical = [c for c in features if (df[c].dtype=='float') or (df[c].dtype=='int') and (c not in categorical)]
########################################################################################################################
#df, periodic = date_encoding(df, time_column, daily=False, weekly=True, yearly=True)
df, norm_deltat, start_date = timediff_encoding(df, time_column)

train = df[df[time_column] < '2015-06-14']
test = df[df[time_column]>='2015-06-14']

X_train, T_train, Y_train, X_test, T_test, Y_test, features, OHEncoders, MS, YScaler = gp_ts_pipeline(train, test, categorical, numerical+norm_deltat, ['delta_t'], scaling_type='log')
o_act, t_act, nt_act, f_act = [X_train.columns.get_loc("Open")], [X_train.columns.get_loc("delta_t")], [X_train.columns.get_loc("norm_delta_t")], [X_train.columns.get_loc(c) for c in features if c not in ["norm_delta_t","delta_t","Open"]]
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
########################################################################################################################
nus, deg = np.asarray([0.5, 1.5, 2.5]), np.asarray([0,1,2,3])
#################### MLE ##########################
def sk_nll_stable(hp):
    done = False
    while not done:
        try:
            kernel = create_kernel_2(hp)
            gpr = GPR(kernel=kernel, optimizer=None).fit(X_train, Y_train)
            done = True
            return hp, -1 * gpr.log_marginal_likelihood_value_
        except (LinAlgError, ValueError):
            traceback.print_exc()
            hp = np.random.uniform(lb, ub, dim)


def mle_objf(pop, dim=None):
    pop = np.clip(pop, lb, ub)
    population, fitness = [], []
    for hp in pop:
        _hp, mse = dask.delayed(sk_nll_stable, nout=2)(hp)
        population.append(_hp), fitness.append(mse)
    population, fitness = dask.compute(population, fitness)
    population, fitness = np.asarray(population), np.asarray(fitness)
    return population, fitness  # duration =  27.99079418182373



def ccf(population):
    return population

lb, ub = 1e-5, 1e3
#lb, ub = [lb]*len(f_act)+[lb]*7+[0], [ub]*len(f_act)+[ub]*7+[3] #create_kernel_2, create_kernel_3
lb, ub = [lb]*len(f_act)+[lb]*11+[0], [ub]*len(f_act)+[ub]*11+[3] #create_kernel_2, create_kernel_3
#lb, ub = [lb]*len(f_act)+[lb]*11, [ub]*len(f_act)+[ub]*11 #create_kernel_4
#lb, ub = [lb]*len(features)+[0, lb, lb], [ub]*len(features)+[3, ub, ub] #create_kernel_5 MATERN, PIECEWISE
#lb, ub = [lb]*len(features)+[lb]*3, [ub]*len(features)+[ub]*3 #create_kernel_5 ARCCOSIN
#lb, ub = [lb]*len(f_act)+[0, lb, lb, lb, lb], [ub]*len(f_act)+[3, ub, ub, ub, ub] #create_kernel_6 MATERN, PIECEWISE
lb, ub = np.asarray(lb), np.asarray(ub)
dim = len(lb)

s = time.time()
optimizer = co.CPSO(mle_objf, ccf, dim, lb, ub, 50, 5000)
e = time.time()
print('duration = ', e - s)
print(optimizer.gbest)
